# Flexible job shop
In this notebook, we show you how to model and solve a flexible job shop problem with PyJobShop.

The Flexible Job Shop Problem (FJSP) is a commonly studied scheduling problem that generalizes many known scheduling problem variants. 
In the FJSP, there is a set of machines and a set of jobs.
Each job is composed of a sequence of tasks, which must be processed in the given sequence.
Each task needs to be processed by exactly one machine that is selected from a set of eligible machines.
The main goal of the FJSP is commonly to minimize the makespan.

## Data

Let's consider a simple example from [Google OR-Tools](https://github.com/google/or-tools/blob/master/examples/python/flexible_job_shop_sat.py). Below we have a data instance with three machines and three jobs, each job consisting of three tasks.

In [ ]:
NUM_MACHINES = 3

# Each job consists of a list of tasks. An task is represented
# by a list of tuples (processing_time, machine), denoting the eligible
# machine assignments and corresponding processing times.
data = [
    [  # Job with three tasks
        [(3, 0), (1, 1), (5, 2)],  # task with three eligible machines
        [(2, 0), (4, 1), (6, 2)],
        [(2, 0), (3, 1), (1, 2)],
    ],
    [
        [(2, 0), (3, 1), (4, 2)],
        [(1, 0), (5, 1), (4, 2)],
        [(2, 0), (1, 1), (4, 2)],
    ],
    [
        [(2, 0), (1, 1), (4, 2)],
        [(2, 0), (3, 1), (4, 2)],
        [(3, 0), (1, 1), (5, 2)],
    ],
]

## Model

In [ ]:
from pyjobshop import Constraint, Model

m = Model()

Data objects such as machines, jobs and tasks can be created with the `Model.add_*` method.

In [ ]:
machines = [
    m.add_machine(name=f"Machine {idx}") for idx in range(NUM_MACHINES)
]

In [ ]:
jobs = {}
tasks = {}

for job_idx, job_data in enumerate(data):
    job = m.add_job(name=f"Job {job_idx}")
    jobs[job_idx] = job

    for idx in range(len(job_data)):
        op_idx = idx = (job_idx, idx)
        tasks[op_idx] = m.add_task(job, name=f"Op {op_idx}")

There are three more things that we need to add to the model:
- tasks have to be associated with their corresponding job;
- Processing times of specific task and machine combinations must be set;
- tasks of the same job must be processed in a given order;

In [ ]:
for job_idx, job_data in enumerate(data):
    for idx, task_data in enumerate(job_data):
        task = tasks[(job_idx, idx)]

        for duration, machine_idx in task_data:
            machine = machines[machine_idx]
            m.add_processing_time(machine, task, duration)

    for idx in range(len(job_data) - 1):
        first = tasks[(job_idx, idx)]
        second = tasks[(job_idx, idx + 1)]
        m.add_constraint(first, second, Constraint.END_BEFORE_START)

Now that we have our model setup correctly, we can solve the model.

In [ ]:
result = m.solve(log=False)
result.status

Turns out we have found the optimal solution! Let's plot that solution.

In [ ]:
from pyjobshop import plot

data = m.data()
plot(data, result.best, plot_labels=True)

## Summary
This concludes the notebook. We showed how to set up an FJSP problem instance using PyJobShop's modeling interface. After setup, we solved the model and plotted the optimal solution.